# 🏠 Dashboard Investissement Locatif - Île-de-France

**Persona ciblé** : Manager IT, 40 ans, cherchant son 1er investissement locatif rentable

**Objectif** : Identifier les meilleures zones pour investir dans un studio/T1/T2 en IDF

**Sources de données** :
- DVF (Demandes de Valeurs Foncières) - Transactions immobilières
- Loyers prédits par commune (data.gouv.fr)
- Accessibilité gares SNCF


In [14]:
import sys
import os
from pathlib import Path
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as W
from IPython.display import display, clear_output, HTML

# Configuration
warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8-darkgrid')

# Ajouter le dossier parent au path pour importer les modules
ROOT_DIR = os.path.abspath(os.path.join(".."))
sys.path.insert(0, ROOT_DIR)

# Import des modules personnalisés
import dvfkit
import data_cleaner

print("✓ Imports réussis")

✓ Imports réussis


In [15]:
# Chargement des données nettoyées (automatique)
df_unifie, df_loyers, df_gares = data_cleaner.quick_load(
    raw_dir="../data/raw",
    clean_dir="../data/clean",
    force_refresh=False  # Mettre True pour forcer le recalcul
)

# Aperçu rapide
print("\n📊 APERÇU DES DONNÉES :")
print(f"   Transactions IDF : {len(df_unifie):,}".replace(",", " "))
print(f"   Communes avec loyers : {len(df_loyers) if df_loyers is not None else 0:,}".replace(",", " "))
print(f"   Gares accessibles : {len(df_gares) if df_gares is not None else 0:,}".replace(",", " "))
print(f"\n   Colonnes disponibles : {list(df_unifie.columns)}")

🧹 NETTOYAGE AUTOMATIQUE DES DONNÉES
✓ DVF déjà nettoyé, chargement...
✓ Loyers déjà nettoyés, chargement...
✓ Gares déjà nettoyées, chargement...

🔗 FUSION DES DATASETS
✓ Fusion loyers : 9 466 lignes enrichies

✅ Nettoyage terminé !
   Dataset final : 56 612 transactions
   Colonnes : date_mutation, valeur_fonciere, surface_reelle_bati, prix_m2, nom_commune, type_local, code_commune_insee, annee...

📊 APERÇU DES DONNÉES :
   Transactions IDF : 56 612
   Communes avec loyers : 1 286
   Gares accessibles : 243

   Colonnes disponibles : ['date_mutation', 'valeur_fonciere', 'surface_reelle_bati', 'prix_m2', 'nom_commune', 'type_local', 'code_commune_insee', 'annee', 'code_postal', 'nom_commune_loyer', 'loyer_m2']


In [16]:
# Initialisation du dashboard
boot = dvfkit.boot_dashboard(
    df=df_unifie,
    theme='light',  # ou 'dark'
    topn=15,
    title="🏠 Investissement Locatif - Île-de-France",
    subtitle="Dashboard interactif pour repérer les meilleures opportunités"
)

# Récupération des contrôles (pour les widgets personnalisés)
controls = dvfkit.get_controls(boot)

print("\n✅ Dashboard lancé ! Utilisez les filtres à gauche pour explorer.")


✅ Dashboard lancé ! Utilisez les filtres à gauche pour explorer.


In [17]:
# Création onglet personnalisé avec 2 widgets côte à côte
out_profil = W.Output()
out_surface = W.Output()

# Ajout onglet au dashboard
onglet_profil = dvfkit.add_tab(boot, "📊 Profil données")
with onglet_profil:
    display(W.HBox([
        W.VBox([out_profil], layout=W.Layout(width='50%', padding='0 8px 0 0')),
        W.VBox([out_surface], layout=W.Layout(width='50%', padding='0 0 0 8px'))
    ]))

def render_widget_profil():
    """Affiche les statistiques du dataset filtré"""
    with out_profil:
        clear_output(wait=True)
        d = dvfkit.apply_filters(df_unifie, controls)
        
        n = len(d)
        na_prix = int(d["prix_m2"].isna().sum())
        
        if d["prix_m2"].notna().any():
            q = d["prix_m2"].quantile([.1, .5, .9]).round(0)
            p10, p50, p90 = int(q.iloc[0]), int(q.iloc[1]), int(q.iloc[2])
        else:
            p10 = p50 = p90 = 0
        
        html = f"""
        <div class="card">
          <h3 style="margin-top:0">📈 Profil du dataset filtré</h3>
          <div style="background:#f0f9ff; padding:12px; border-radius:8px; margin:8px 0">
            <div><b>Taille :</b> {n:,} transactions</div>
            <div><b>Prix/m² manquants :</b> {na_prix:,}</div>
          </div>
          <div style="background:#fef3c7; padding:12px; border-radius:8px; margin:8px 0">
            <div><b>Déciles prix/m² :</b></div>
            <div>• 10e centile : {p10:,} €</div>
            <div>• Médiane (50e) : {p50:,} €</div>
            <div>• 90e centile : {p90:,} €</div>
          </div>
        </div>
        """.replace(",", " ")
        display(HTML(html))

def render_widget_surface():
    """Histogramme des surfaces"""
    with out_surface:
        clear_output(wait=True)
        d = dvfkit.apply_filters(df_unifie, controls)
        
        if len(d) and d["surface_reelle_bati"].notna().any():
            s = d["surface_reelle_bati"].dropna()
            
            plt.figure(figsize=(6.5, 4))
            plt.hist(s, bins=40, color='steelblue', edgecolor='white', alpha=0.8)
            plt.title("Distribution des surfaces (m²)", fontsize=14, fontweight='bold')
            plt.xlabel("Surface (m²)", fontsize=11)
            plt.ylabel("Fréquence", fontsize=11)
            plt.grid(alpha=0.3, linestyle='--')
            plt.tight_layout()
            plt.show()

# Observers
for key in ["w_surface", "w_loyer", "w_topn", "w_iqr", "w_commune"]:
    if key in controls:
        controls[key].observe(lambda _: (render_widget_profil(), render_widget_surface()), "value")

if isinstance(controls.get("w_year"), W.SelectionRangeSlider):
    controls["w_year"].observe(lambda _: (render_widget_profil(), render_widget_surface()), "value")

# Rendu initial
render_widget_profil()
render_widget_surface()

In [18]:
out_scatter = W.Output()
out_prix_stats = W.Output()

onglet_prix = dvfkit.add_tab(boot, "💰 Prix/m²")
with onglet_prix:
    display(W.HBox([
        W.VBox([out_scatter], layout=W.Layout(width='60%', padding='0 8px 0 0')),
        W.VBox([out_prix_stats], layout=W.Layout(width='40%', padding='0 0 0 8px'))
    ]))

def render_widget_scatter():
    """Nuage de points prix/m² vs surface"""
    with out_scatter:
        clear_output(wait=True)
        d = dvfkit.apply_filters(df_unifie, controls)
        
        if len(d) and {"surface_reelle_bati", "prix_m2"}.issubset(d.columns):
            ds = d.dropna(subset=["surface_reelle_bati", "prix_m2"])
            ds = ds.sample(min(4000, len(ds)), random_state=42)
            
            if len(ds):
                plt.figure(figsize=(7.5, 4.5))
                plt.scatter(ds["surface_reelle_bati"], ds["prix_m2"], 
                           s=20, alpha=0.5, c='coral', edgecolors='white', linewidth=0.5)
                plt.title("Prix/m² selon la surface", fontsize=14, fontweight='bold')
                plt.xlabel("Surface (m²)", fontsize=11)
                plt.ylabel("Prix/m² (€)", fontsize=11)
                plt.grid(alpha=0.3, linestyle='--')
                plt.tight_layout()
                plt.show()

def render_widget_prix_stats():
    """Statistiques prix par tranche de surface"""
    with out_prix_stats:
        clear_output(wait=True)
        d = dvfkit.apply_filters(df_unifie, controls)
        
        if len(d):
            # Catégorisation
            bins = [0, 30, 45, 65, 200]
            labels = ['Studio/T1', 'T2', 'T3', 'T4+']
            d['categorie'] = pd.cut(d['surface_reelle_bati'], bins=bins, labels=labels, include_lowest=True)
            
            stats = d.groupby('categorie', observed=True).agg({
                'prix_m2': ['count', 'median', 'mean'],
                'valeur_fonciere': 'median'
            }).round(0)
            
            stats.columns = ['Nb', 'Prix/m² méd.', 'Prix/m² moy.', 'Prix total méd.']
            
            html = stats.to_html(classes='table table-striped')
            display(HTML(f"""
            <div class="card">
                <h4 style="margin-top:0">📊 Stats par type de bien</h4>
                {html}
            </div>
            """))

# Observers
for key in ["w_surface", "w_loyer", "w_topn", "w_iqr", "w_commune"]:
    if key in controls:
        controls[key].observe(lambda _: (render_widget_scatter(), render_widget_prix_stats()), "value")

if isinstance(controls.get("w_year"), W.SelectionRangeSlider):
    controls["w_year"].observe(lambda _: (render_widget_scatter(), render_widget_prix_stats()), "value")

render_widget_scatter()
render_widget_prix_stats()

In [19]:
if df_loyers is not None and 'loyer_m2' in df_unifie.columns:
    
    out_rendement = W.Output()
    out_top_rendement = W.Output()
    
    onglet_rendement = dvfkit.add_tab(boot, "🎯 Rendement")
    with onglet_rendement:
        display(W.HBox([
            W.VBox([out_rendement], layout=W.Layout(width='55%', padding='0 8px 0 0')),
            W.VBox([out_top_rendement], layout=W.Layout(width='45%', padding='0 0 0 8px'))
        ]))
    
    def render_widget_rendement():
        """Histogramme des rendements bruts"""
        with out_rendement:
            clear_output(wait=True)
            d = dvfkit.apply_filters(df_unifie, controls)
            
            if len(d) and d["yield_brut"].notna().any():
                y = (d["yield_brut"] * 100).clip(upper=15)  # Cap à 15% pour lisibilité
                
                plt.figure(figsize=(7, 4.2))
                plt.hist(y, bins=50, color='mediumseagreen', edgecolor='white', alpha=0.85)
                plt.axvline(y.median(), color='red', linestyle='--', linewidth=2, 
                           label=f'Médiane : {y.median():.1f}%')
                plt.title("Distribution des rendements bruts", fontsize=14, fontweight='bold')
                plt.xlabel("Rendement brut (%)", fontsize=11)
                plt.ylabel("Nombre de biens", fontsize=11)
                plt.legend()
                plt.grid(alpha=0.3, linestyle='--')
                plt.tight_layout()
                plt.show()
    
    def render_widget_top_rendement():
        """Top communes par rendement"""
        with out_top_rendement:
            clear_output(wait=True)
            d = dvfkit.apply_filters(df_unifie, controls)
            
            if len(d):
                top = (d.groupby(["nom_commune", "code_postal"], as_index=False)
                       .agg(rendement_med=("yield_brut", "median"),
                            nb=("yield_brut", "count"))
                       .sort_values("rendement_med", ascending=False)
                       .head(10))
                
                top["rendement_med"] = (top["rendement_med"] * 100).round(2)
                top = top.rename(columns={
                    "nom_commune": "Commune",
                    "code_postal": "CP",
                    "rendement_med": "Yield (%)",
                    "nb": "Nb ventes"
                })
                
                html = top.style.hide(axis="index") \
                          .background_gradient(subset=["Yield (%)"], cmap='YlGn') \
                          .format({"Yield (%)": "{:.2f}%"}) \
                          .to_html()
                
                display(HTML(f"""
                <div class="card">
                    <h4 style="margin-top:0">🏆 Top 10 rendements</h4>
                    {html}
                    <p style="color:#666; font-size:0.9em; margin-top:8px">
                    💡 Basé sur loyer/m² médian de chaque commune
                    </p>
                </div>
                """))
    
    # Observers
    for key in ["w_surface", "w_loyer", "w_topn", "w_iqr", "w_commune"]:
        if key in controls:
            controls[key].observe(lambda _: (render_widget_rendement(), render_widget_top_rendement()), "value")
    
    if isinstance(controls.get("w_year"), W.SelectionRangeSlider):
        controls["w_year"].observe(lambda _: (render_widget_rendement(), render_widget_top_rendement()), "value")
    
    render_widget_rendement()
    render_widget_top_rendement()
    
else:
    print("⚠️ Données loyers non disponibles - Widget rendement désactivé")

In [20]:
if df_gares is not None:
    
    out_gares = W.Output()
    
    onglet_gares = dvfkit.add_tab(boot, "🚆 Gares")
    with onglet_gares:
        display(W.VBox([out_gares]))
    
    def render_widget_gares():
        """Top gares accessibles"""
        with out_gares:
            clear_output(wait=True)
            
            if 'niveau_max' in df_gares.columns:
                top_gares = df_gares.head(20)
                
                plt.figure(figsize=(10, 6))
                plt.barh(top_gares['nom_gare'], top_gares['niveau_max'], 
                        color='dodgerblue', edgecolor='white', alpha=0.85)
                plt.xlabel("Niveau d'accessibilité (max)", fontsize=11)
                plt.title("Top 20 gares les plus accessibles - IDF", fontsize=14, fontweight='bold')
                plt.gca().invert_yaxis()
                plt.grid(alpha=0.3, axis='x', linestyle='--')
                plt.tight_layout()
                plt.show()
                
                html_info = """
                <div class="card">
                    <p><b>💡 Niveaux d'accessibilité :</b></p>
                    <ul>
                        <li><b>1-2</b> : Accessibilité limitée</li>
                        <li><b>3</b> : Accessibilité partielle</li>
                        <li><b>4-5</b> : Bonne accessibilité (PMR)</li>
                    </ul>
                </div>
                """
                display(HTML(html_info))
    
    render_widget_gares()
    
else:
    print("⚠️ Données gares non disponibles - Widget accessibilité désactivé")

In [21]:
out_carte = W.Output()

onglet_carte = dvfkit.add_tab(boot, "🗺️ Carte départements")
with onglet_carte:
    display(W.VBox([out_carte]))

def render_widget_carte():
    """Carte prix/m² par département"""
    with out_carte:
        clear_output(wait=True)
        d = dvfkit.apply_filters(df_unifie, controls)
        
        if len(d) and 'code_departement' in d.columns:
            dept_stats = (d.groupby('code_departement')
                          .agg(prix_med=('prix_m2', 'median'),
                               nb=('prix_m2', 'count'))
                          .sort_values('prix_med', ascending=False)
                          .head(8))
            
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))
            
            # Prix médian
            ax1.barh(dept_stats.index, dept_stats['prix_med'], 
                    color='orangered', edgecolor='white', alpha=0.85)
            ax1.set_xlabel("Prix/m² médian (€)", fontsize=11)
            ax1.set_title("Prix médian par département", fontsize=12, fontweight='bold')
            ax1.invert_yaxis()
            ax1.grid(alpha=0.3, axis='x', linestyle='--')
            
            # Nombre de ventes
            ax2.barh(dept_stats.index, dept_stats['nb'], 
                    color='mediumseagreen', edgecolor='white', alpha=0.85)
            ax2.set_xlabel("Nombre de transactions", fontsize=11)
            ax2.set_title("Volume de ventes par département", fontsize=12, fontweight='bold')
            ax2.invert_yaxis()
            ax2.grid(alpha=0.3, axis='x', linestyle='--')
            
            plt.tight_layout()
            plt.show()

# Observers
for key in ["w_surface", "w_loyer", "w_topn", "w_iqr", "w_commune"]:
    if key in controls:
        controls[key].observe(lambda _: render_widget_carte(), "value")

if isinstance(controls.get("w_year"), W.SelectionRangeSlider):
    controls["w_year"].observe(lambda _: render_widget_carte(), "value")

render_widget_carte()

In [22]:
# Calcul des insights pour le persona
d = dvfkit.apply_filters(df_unifie, controls)

if len(d):
    prix_med = d["prix_m2"].median()
    surface_med = d["surface_reelle_bati"].median()
    prix_total_med = d["valeur_fonciere"].median()
    
    if "yield_brut" in d.columns and d["yield_brut"].notna().any():
        yield_med = d["yield_brut"].median() * 100
    else:
        yield_med = None
    
    # Top 3 communes
    top3 = (d.groupby("nom_commune", as_index=False)
            .agg(nb=("prix_m2", "count"),
                 prix_m2_med=("prix_m2", "median"))
            .sort_values("nb", ascending=False)
            .head(3))
    
    print("\n" + "="*70)
    print("🎯 RECOMMANDATIONS POUR VOTRE INVESTISSEMENT")
    print("="*70)
    print(f"\n📊 Profil de la sélection actuelle :")
    print(f"   • Surface médiane : {surface_med:.0f} m²")
    print(f"   • Prix/m² médian : {prix_med:,.0f} €".replace(",", " "))
    print(f"   • Prix total médian : {prix_total_med:,.0f} €".replace(",", " "))
    
    if yield_med:
        print(f"   • Rendement brut médian : {yield_med:.2f}%")
        
        if yield_med >= 5.5:
            print("\n   ✅ EXCELLENT rendement pour un investissement locatif IDF !")
        elif yield_med >= 4.5:
            print("\n   ✔️ BON rendement, conforme au marché IDF")
        else:
            print("\n   ⚠️ Rendement modéré - Privilégiez la valorisation long terme")
    
    print(f"\n🏆 Top 3 communes les plus actives :")
    for i, row in top3.iterrows():
        print(f"   {i+1}. {row['nom_commune']} - {row['nb']} ventes ({row['prix_m2_med']:,.0f} €/m²)".replace(",", " "))
    
    print("\n💡 Conseils :")
    if surface_med < 35:
        print("   • Studio/T1 : Idéal pour étudiants, forte demande locative")
    elif surface_med < 50:
        print("   • T2 : Bon compromis rendement/valorisation")
    else:
        print("   • T3+ : Cible familles/colocation, valorisation long terme")
    
    print("\n   • Vérifiez la proximité transports (RER/métro)")
    print("   • Privilégiez les quartiers estudiantins ou pôles d'emploi")
    print("   • Prévoyez 20-25% du loyer pour charges/vacance/travaux")
    
    print("\n📁 N'oubliez pas d'exporter vos résultats (bouton en haut) !")
    print("="*70)


🎯 RECOMMANDATIONS POUR VOTRE INVESTISSEMENT

📊 Profil de la sélection actuelle :
   • Surface médiane : 51 m²
   • Prix/m² médian : 5 103 €
   • Prix total médian : 245 000 €
   • Rendement brut médian : 5.17%

   ✔️ BON rendement, conforme au marché IDF

🏆 Top 3 communes les plus actives :
   649. PARIS 18 - 972 ventes (8 513 €/m²)
   646. PARIS 15 - 817 ventes (9 286 €/m²)
   648. PARIS 17 - 699 ventes (9 744 €/m²)

💡 Conseils :
   • T3+ : Cible familles/colocation, valorisation long terme

   • Vérifiez la proximité transports (RER/métro)
   • Privilégiez les quartiers estudiantins ou pôles d'emploi
   • Prévoyez 20-25% du loyer pour charges/vacance/travaux

📁 N'oubliez pas d'exporter vos résultats (bouton en haut) !
